Attempting to re-create plots from before code refactor. These plots are originally from `Model.ipynb` in the original `hyperspectral` repository.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_objects import Fluorophore
from imaging_model import fast_form_A
#from information_matrix import read_qe, fast_form_q_vec, FIM

Get fluorophore data: (copied from data_objects.py to help future-proof code)

In [2]:
fluorophore_string_list = ["mEmerald", "mTagBFP2", "mCherry", "mNeptune2.5"]
brightness_list         = [39.1,       32.38,      15.85,     22.8         ]
wavelength_range = (400,900)

fluorophore_list = []
for fluorophore_name, fluorophore_brightness in zip(fluorophore_string_list, brightness_list):
    fluorophore_list.append(Fluorophore(fluorophore_name, fluorophore_brightness, index_range=wavelength_range))

list(map(Fluorophore.get_name, fluorophore_list))

['mEmerald', 'mTagBFP2', 'mCherry', 'mNeptune2.5']

Specify other parameters:

In [3]:
illumination_wavelengths = np.array([405, 488, 561, 637])
k = np.array([1,1,1,1]) # we will first compute A without k, and then use A to find new k values
bin_width = 10

params_wo_k = (illumination_wavelengths, k, wavelength_range, bin_width, fluorophore_list)

Imaging model before $k$ is specified:

In [4]:
fast_form_A(*params_wo_k)[10,0]

0.017846902914808855

Now, we want to balance out our emission strengths so that a similar amount of information (photons) is collected from each fluorophore. (copied from original code)

In [5]:
def calc_k_strength(i, desired_photons):
        A_constant_k = fast_form_A(*params_wo_k)
        illumination_one = A_constant_k[0:50, 0:4]
        illumination_two = A_constant_k[50:100, 0:4]
        illumination_three = A_constant_k[100:150, 0:4]
        illumination_four = A_constant_k[150:200, 0:4]
        if 0 <= i < 50: 
                sliced_average = np.average(illumination_one)
                k = 1/(sliced_average) * desired_photons
        elif 50 <= i < 100:
                sliced_average = np.average(illumination_two)
                k = 1/(sliced_average) *desired_photons
        elif 100 <= i < 150:
                sliced_average = np.average(illumination_three)
                k = 1/(sliced_average) * desired_photons
        elif 150 <= i < 200:
                sliced_average = np.average(illumination_four)
                k = 1/(sliced_average) * desired_photons
        return k

In [6]:
calc_k_strength(10, 100)

30740.88858655248

Imaging model with k specified: (copied from original code)

In [7]:
def form_k_list(desired_photons):
    k_list = []
    for i in [0, 50, 100, 150]:
        k = calc_k_strength(i, desired_photons)
        k_list.append(k)
        
    return k_list

def form_matrix_A(desired_photons):
    k_list = form_k_list(desired_photons)
    return fast_form_matrix_A(illumination_wavelengths, k_list, wavelength_range, bin_width, fluorophore_list)

Created a list of matrices so we are now able to create multiple "A" matrices for desired photon values. (copied from original code)

In [8]:
def calc_A_for_varying_photons(desired_photons_list):
    # Input: list of average detected photons desired
    # Output: list of corresponding A matrices
    matrix_A_list = []
    for desired_photons in desired_photons_list:
        A = form_matrix_A(desired_photons)
        matrix_A_list.append(A)

    return matrix_A_list

Create a vector consisting of some concentration of each fluorophore (copied from original code)

In [9]:
x = [1, 1, 1, 1]

Function to calculate FOM from $A$ and $x$.

In [10]:
def calc_FOM(A, x):
    y = A @ x
    y_inv = 1/y
    y_inv[y_inv == np.inf] = 0
    FIM = (A.T * y_inv) @ A
    FIM_inv = np.linalg.inv(FIM)
    CRLB = np.diagonal(FIM_inv)
    FOM_list = np.array(x) / np.sqrt(CRLB)
    
    return FOM_list

Function to determine average photons detected using a for-loop (copied from original code)

In [11]:
def calc_avg_detected_photons(A):
    avg_photons_detected_list = []
    for illum_count in range(4):
        # start = 50 * illum_count
        # end = 50 * (illum_count + 1)
        # print(f"Start row: {start}, End row: {end}")
        A_section = A[50 * illum_count : 50 * (illum_count + 1), :]
        avg_photons_detected = np.average(A_section)
        avg_photons_detected_list.append(avg_photons_detected)

    return avg_photons_detected_list